In [ ]:
#intalling required packages 
! pip install mysql-connector-python
! pip install pandas
! pip install requests
! pip install tabulate

#importing required fun
import pandas as pd
import mysql.connector as mysql

# creating connection to database 
mydb = mysql.connect( 
    host = "localhost",
    user= "root",
    password= "",
    port=3306 )

#selecting database file 

mycursor.execute("use guvi")

# getting file through API

import requests

url = "https://api.sportradar.com/tennis/trial/v3/en/competitions.json?api_key=6c7tsnpkhnfoZT5MQ1ZzJeF0L8u4P1g4Bw89SiQ2"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.text)


# saving joson file 

import json
data = response.json()
with open("competitions.json", "w") as json_file:
    json.dump(data, json_file, indent=4)

# creating dataframe table on pandas by converting json file

with open("competitions.json", "r") as f:
    data = json.load(f)

competitions_df= pd.json_normalize(data["competitions"])
category_df = competitions_df[["category.id", "category.name"]]
competitions_df = competitions_df.drop(columns=[ "category.name"])
category_df = category_df.drop_duplicates(subset='category.id')
competitions_df = competitions_df.where(pd.notnull(competitions_df), None)


# connecting mysql

mycursor = mydb.cursor(buffered = True)

%pip install tabulate
from tabulate import tabulate


# creating  competition and category table in SQL

mycursor.execute("create table category (category_id VARCHAR(50) PRIMARY KEY, category_name VARCHAR(100)not null)")

mycursor.execute('''create table competitions (competition_id VARCHAR(50) PRIMARY KEY, 
competition_name VARCHAR(100)not null,
parent_id varchar(50),
type varchar(20) not null,
gender varchar(10) not null,
category_id VARCHAR(50), 
FOREIGN KEY (category_id)REFERENCES category (category_id))''')

# inserting value from pd data frame

insert = "insert into category (category_id,category_name) values(%s,%s)"
values_to_insert =[tuple(x) for x in category_df[['category.id','category.name']].values]
mycursor.executemany(insert,values_to_insert)
mydb.commit()


insert = "insert into competitions (competition_id,competition_name,parent_id,type,gender,category_id) values(%s,%s,%s,%s,%s,%s)"
values_to_insert =[tuple(x) for x in competitions_df [['id','name','parent_id','type','gender','category.id']].values]
mycursor.executemany(insert,values_to_insert)
mydb.commit()



#1)List all competitions along with their category name

mycursor.execute("select competitions.*,category.category_name from competitions join category on competitions.category_id= category.category_id")
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

#2)Count the number of competitions in each category

mycursor.execute('''SELECT count(competition_name),category.category_name 
FROM Competitions 
JOIN category on competitions.category_id= category.category_id GROUP BY category.category_name ''')
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

#3)Find all competitions of type 'doubles'

mycursor.execute("SELECT competition_name FROM competitions WHERE type ='doubles'")
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

#4) Get competitions that belong to a specific category (e.g., ITF Men)

mycursor.execute('''select competitions.competition_name,category.category_name 
from competitions 
join category on competitions.category_id= category.category_id 
order by category_name''') # to filter specific category we can use where category_name = 'ITF men'etc.... instead of order by
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

#5) Identify parent competitions and their sub-competitions

mycursor.execute("select parent_id,competition_id as Sub_ID ,competition_name as sub_competitions from competitions where parent_id = parent_id order by parent_id ")
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

#6)Analyze the distribution of competition types by category


mycursor.execute('''select category.category_name,competitions.competition_name ,competitions.type 
from competitions 
join category on competitions.category_id= category.category_id 
order by category_name''')
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

#7) List all competitions with no parent (top-level competitions)


mycursor.execute("SELECT competitions.* FROM competitions WHERE parent_id IS NULL ")
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))



# TABLE 2
#connecting API

import requests

url = "https://api.sportradar.com/tennis/trial/v3/en/complexes.json?api_key=6c7tsnpkhnfoZT5MQ1ZzJeF0L8u4P1g4Bw89SiQ2"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.text)

# saving json 

import json
data = response.json()
with open("COMPLEXES DATA.json", "w") as json_file:
    json.dump(data, json_file, indent=4)

#json to pandas DF 

with open("COMPLEXES DATA.json", "r") as f:
    data = json.load(f)

    complex_df = pd.json_normalize(data['complexes']) [['id','name']]
complexes_with_venues = [c for c in data['complexes'] if 'venues' in c and c['venues']]
venues_df = pd.json_normalize(
    complexes_with_venues,
    record_path='venues',
    meta=['id'],
    meta_prefix='complex_'
)



# creating complexes and venues table 
#complex
mycursor.execute("create table complexes (complex_id VARCHAR(50) primary key,complex_name VARCHAR(100) not null) ")

#venues
mycursor.execute(''' create table Venues  (venue_id VARCHAR(50) PRIMARY KEY ,
                 venue_name VARCHAR(100) not null,
                 city_name VARCHAR(100) not null,
                 country_name VARCHAR(100) not null,
                 country_code char(3) not null,
                 timezone VARCHAR(100) not null,
                 complex_id VARCHAR(50), FOREIGN KEY (complex_id) REFERENCES complexes (complex_id))''')


#insering values in complex and venues table from pandas df 
#complex

insert = "insert into complexes (complex_id,complex_name) values(%s,%s)"
values_to_insert =[tuple(x) for x in venues_df [['id','name']].values]
mycursor.executemany(insert,values_to_insert)
mydb.commit()


#venues 

insert = "insert into Venues (venue_id,venue_name,city_name,country_name,country_code,timezone,complex_id) values(%s,%s,%s,%s,%s,%s,%s)"
values_to_insert =[tuple(x) for x in venues_df [['id','name','city_name','country_name','country_code','timezone','complex_id']].values]
mycursor.executemany(insert,values_to_insert)
mydb.commit()



#1)List all venues along with their associated complex name

mycursor.execute("select Venues.*,complexes.complex_name from Venues join complexes on Venues.complex_id = complexes.complex_id")
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

#2) Count the number of venues in each complex
mycursor.execute('''select complexes.complex_name,count(venue_name) as no_of_venues 
from Venues 
join complexes on Venues.complex_id = complexes.complex_id group by complex_name''')

out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

#3)Get details of venues in a specific country (e.g., Chile)
mycursor.execute("select * from venues where country_name ='Chile') # istead of chile you can use whatever country name you want .
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))


#4)Identify all venues and their timezones
mycursor.execute("select venue_name,timezone from venues")
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

#5)Find complexes that have more than one venue

mycursor.execute('''select complexes.*,count(venue_id) as no_of_venues 
from Venues 
join complexes on Venues.complex_id = complexes.complex_id 
group by complex_name,complex_id having count(venue_id) >1 ''')
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

#6) List venues grouped by country
mycursor.execute(" select * from venues order by country_name")
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

#7) Find all venues for a specific complex (e.g., Nacional)

mycursor.execute('''select venues.*,complexes.complex_name
from Venues 
join complexes on Venues.complex_id = complexes.complex_id  
where complex_name = 'Nacional' ''')
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))



#TABLE 3

#connecting API 

import requests

url = "https://api.sportradar.com/tennis/trial/v3/en/double_competitors_rankings.json?api_key=6c7tsnpkhnfoZT5MQ1ZzJeF0L8u4P1g4Bw89SiQ2"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.text)


#saving Json 


import json
data = response.json()
with open("DOUBLES COMPETITOR RANKINGS.json", "w") as json_file:
    json.dump(data, json_file, indent=4)

# json to DF

with open("DOUBLES COMPETITOR RANKINGS.json", "r") as f:
    data = json.load(f)


Competitor_info_df = pd.json_normalize(
    data['rankings'],
    record_path='competitor_rankings')
Competitor_info_df = Competitor_info_df.fillna('NULL')



# creating Competitors  and Competitor_Rankings table in SQL on given format.

mycursor.execute('''create table Competitors (competitor_id VARCHAR(50) PRIMARY KEY,
                 name VARCHAR(100) NOT NULL,
                 country VARCHAR(100) NOT NULL,
                 country_code CHAR(3) NOT NULL,
                 abbreviation VARCHAR(10) NOT NULL)''')


mycursor.execute('''create table Competitor_Rankings (rank_id INT PRIMARY KEY AUTO_INCREMENT,
                 rank INT NOT NULL,
                 movement INT NOT NULL,
                 points INT NOT NULL,
                 competitions_played INT NOT NULL,
                 competitor_id VARCHAR(50),FOREIGN KEY (competitor_id) REFERENCES Competitors  (competitor_id))''')


#storing values from DF to data tabeles 

# for Competitors table 

insert = "insert into Competitors (competitor_id,name,country,country_code,abbreviation) values(%s,%s,%s,%s,%s)"
values_to_insert =[tuple(x) for x in Competitor_info_df [['competitor.id','competitor.name','competitor.country','competitor.country_code','competitor.abbreviation']].values]
mycursor.executemany(insert,values_to_insert)
mydb.commit()

# for Competitor_Rankings table 

insert = "insert into Competitor_Rankings (rank,movement,points,competitions_played,competitor_id) values(%s,%s,%s,%s,%s)"
values_to_insert =[tuple(x) for x in Competitor_info_df [['rank','movement','points','competitions_played','competitor.id' ]].values]
mycursor.executemany(insert,values_to_insert)
mydb.commit()



#1) Get all competitors with their rank and points.


mycursor.execute('''select Competitors.*,Competitor_Rankings.rank,Competitor_Rankings.points
from Competitors
join Competitor_Rankings on Competitors.competitor_id = Competitor_Rankings.competitor_id ''')

out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))


#2)Find competitors ranked in the top 5

mycursor.execute('''select Competitors.*,Competitor_Rankings.rank
from Competitors
join Competitor_Rankings on Competitors.competitor_id = Competitor_Rankings.competitor_id  
where rank <=5 ''')

out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

#3) List competitors with no rank movement (stable rank)

mycursor.execute('''select Competitor_Rankings.movement,Competitors.*
from Competitors
join Competitor_Rankings on Competitors.competitor_id = Competitor_Rankings.competitor_id  
where movement =0 ''')
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))


#4)Get the total points of competitors from a specific country (e.g., Croatia)

mycursor.execute('''select sum(Competitor_Rankings.points) as total_points 
from Competitors
join Competitor_Rankings on Competitors.competitor_id = Competitor_Rankings.competitor_id  
where country =  'Croatia' ''')


out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))


#5) Count the number of competitors per country

mycursor.execute(" select count(name) as No_of_competitors ,country from Competitors group by country ")


out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))


#6)Find competitors with the highest points in the current week

mycursor.execute('''select max(Competitor_Rankings.points) as highest_points,Competitors.*
from Competitors
join Competitor_Rankings on Competitors.competitor_id = Competitor_Rankings.competitor_id ''')

out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))





